In [ ]:
import pandas as pd
import os
import re


carpeta = ""
ruta_archivo = os.path.join(carpeta, "dataset_02-2026.csv")

df_sucio = []
df_sucio = pd.read_csv(ruta_archivo)

# Imprimimos el head
print("-" * 30)
print(df_sucio.dtypes)

------------------------------
id                        object
site                      object
job_url                   object
job_url_direct            object
title                     object
company                   object
location                  object
date_posted               object
job_type                  object
salary_source             object
interval                  object
min_amount               float64
max_amount               float64
currency                  object
is_remote                   bool
job_level                 object
job_function              object
listing_type              object
emails                    object
description               object
company_industry          object
company_url               object
company_logo              object
company_url_direct        object
company_addresses         object
company_num_employees     object
company_revenue           object
company_description       object
skills                   float64
experience_r

In [12]:
df_clean = df_sucio.copy()

In [13]:
# Lista de columnas a eliminar
cols_to_drop = ['job_url', 'company_url', 'company_logo']

# Usamos errors='ignore' para que no rompa el código si la columna ya fue borrada
df_clean.drop(columns=cols_to_drop, inplace=True, errors='ignore')

# Opcional: Limpiar automáticamente columnas que estén totalmente vacías (NaN)
df_clean.dropna(axis=1, how='all', inplace=True)

print(df_clean.columns)

Index(['id', 'site', 'job_url_direct', 'title', 'company', 'location',
       'date_posted', 'job_type', 'salary_source', 'interval', 'min_amount',
       'max_amount', 'currency', 'is_remote', 'job_level', 'job_function',
       'listing_type', 'emails', 'description', 'company_industry',
       'company_url_direct', 'company_addresses', 'company_num_employees',
       'company_revenue', 'company_description', 'search_location',
       'search_query'],
      dtype='object')


In [14]:
mapeoTipos = {
    'id' : 'string',
    'title' : 'string',
    'company' : 'string',
    'location' : 'string',
    'job_type' : 'string',
    'job_level' : 'string',
    'job_function' : 'string',
    'description' : 'string',
    'company_industry' : 'string',
    'search_location' : 'string'
    }

df_clean = df_clean.astype(mapeoTipos)

In [ ]:
def detectar_stack(descripcion):
    if pd.isna(descripcion) or descripcion == "":
        return "no especificado"
    
    found = []
    desc_lower = str(descripcion).lower()
    
    # Lista ampliada de tecnologías en minúsculas
    tech_keywords = [
        # Lenguajes
        'python', 'sql', 'java', 'scala', 'c#', 'c++', 'go', 'rust', 'typescript', 'javascript', 
        'php', 'ruby', 'swift', 'kotlin', 'dart', 'elixir', 'perl', 'r', 'shell', 'bash',
        
        # Cloud & DevOps
        'aws', 'azure', 'gcp', 'google cloud', 'docker', 'kubernetes', 'terraform', 'ansible', 
        'jenkins', 'git', 'ci/cd', 'github', 'gitlab', 'bitbucket', 'linux', 'unix', 'vagrant',
        'prometheus', 'grafana', 'elk', 'splunk', 'circleci', 'travisci', 'helm',
        
        # Big Data & Data Engineering
        'spark', 'kafka', 'hadoop', 'flink', 'airflow', 'snowflake', 'databricks', 'bigquery', 
        'redshift', 'etl', 'elt', 'dbt', 'airbyte', 'nifi', 'hive', 'pig', 'clickhouse', 
        'presto', 'trino', 'looker', 'tableau', 'power bi', 'superset',
        
        # Backend & Frameworks
        'node.js', 'express', 'django', 'flask', 'fastapi', 'spring', 'hibernate', 'laravel', 
        'symfony', 'rails', '.net', 'asp.net', 'microservices', 'api', 'graphql', 'rest',
        
        # Frontend & Mobile
        'react', 'angular', 'vue', 'next.js', 'nuxt.js', 'svelte', 'tailwind', 'bootstrap', 
        'flutter', 'react native', 'ionic', 'xamarin',
        
        # Bases de Datos
        'postgresql', 'mysql', 'oracle', 'mongodb', 'redis', 'elasticsearch', 'cassandra', 
        'dynamodb', 'firebase', 'mariadb', 'sqlite', 'nosql', 'couchdb', 'neo4j',
        
        # IA & Ciencia de Datos
        'pandas', 'numpy', 'scikit-learn', 'tensorflow', 'pytorch', 'keras', 'opencv', 
        'huggingface', 'scipy', 'matplotlib', 'seaborn'
    ]

    for tech in tech_keywords:
        # Usamos regex para palabras exactas
        pattern = r'\b' + re.escape(tech) + r'\b'
        
        # Ajuste específico para lenguajes con símbolos o versiones con punto
        if tech in ['c++', 'c#']:
            pattern = re.escape(tech) + r'(?!\w)'
        elif tech == 'node.js':
            pattern = r'node\.js\b|nodejs\b' # Detecta 'node.js' o 'nodejs'
        elif tech == '.net':
            pattern = r'\.net\b'

        if re.search(pattern, desc_lower):
            found.append(tech)
            
    return ", ".join(sorted(list(set(found)))) if found else "no especificado"

In [16]:
df_clean['job_url_direct'] = df_clean['job_url_direct'].notna()

df_clean['date_posted'] = pd.to_datetime(df_clean['date_posted'], errors='coerce')

df_clean['emails'] = df_clean['emails'].notna()

In [ ]:
df_clean['stack_tecnologico'] = df_clean['description'].apply(detectar_stack)

TypeError: boolean value of NA is ambiguous

In [ ]:
df_clean['site'].value_counts()

site
linkedin     1141
glassdoor     267
indeed          3
Name: count, dtype: int64

In [ ]:
columnas_finales = df_clean.columns.tolist()

for columna in [col for col in columnas_finales if col not in ['description','id']]:
    print(f"Columna: {columna}")
    print(f"Valores únicos: {df_clean[columna].value_counts()}")

Columna: site
Valores únicos: site
linkedin     1141
glassdoor     267
indeed          3
Name: count, dtype: int64
Columna: job_url_direct
Valores únicos: job_url_direct
False    1113
True      298
Name: count, dtype: int64
Columna: title
Valores únicos: title
data engineer                                                            60
data analyst                                                             33
software engineer                                                        32
full stack developer                                                     14
graduate software engineer                                               12
                                                                         ..
programador java /jenkins/sonar                                           1
java developer - spain                                                    1
junior software developer – dashboards y apis                             1
programadores junior/senior java                       

In [ ]:
ruta_salida = os.path.join(carpeta, "dataset_limpio.csv")
df_clean.to_csv(ruta_salida, index=False)